In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [4]:
#import dataframe
path=r'C:\\Users\\Admin\\London Uk Airbnb'

In [5]:
## This command propts matplotlib visuals to appear in the notebook 
%matplotlib inline

In [1]:
Country_geo =  r'C:\Users\Admin\London Uk Airbnb\Original data\Uk_city.geo.json'

In [6]:
f = open(r'C:\Users\Admin\London Uk Airbnb\Original data\Uk_city.geo.json',)
  
# returns JSON object asa dictionary
data = json.load(f)
  
# Iterating through the json list
for i in data['features']:
    print(i)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 2245: character maps to <undefined>

In [11]:
df = pd.read_csv(os.path.join(path,'Prepared data','data_new.csv'))
df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_12412\2743392227.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(path,'Prepared data','data_new.csv'))


,Unnamed: 0,Id,Name,Host_id,Neighbourhood,Room_type,price,Min_nights,Last_review,Calculated_listing_count,Availability_365,Nr_of_reviews_ltm
0,50,41870,Room in relaxed family house!,182993,Waltham Forest,Private room,37,2,2011-07-02,1,0,0
1,372,388306,"Garden flat, Putney",1942893,Wandsworth,Private room,908,1,2012-03-26,1,365,0
2,66,306405,Spacious Double Room in East London,1575800,Barking and Dagenham,Private room,41,4,2012-05-27,1,365,0
3,435,390876,Peaceful Riverside Island Sanctuary,1954980,Richmond upon Thames,Private room,35,3,2012-05-31,1,0,0
4,799,553543,Newly furnished flat in Kensington,2720301,Hammersmith and Fulham,Private room,35,14,2012-07-17,1,0,0


In [12]:
# Drop the redundant columns
df = df.drop(['Unnamed: 0'],axis = 1)

# 2. Data Wrangling

In [16]:
#create a new column for total view for each country

df['Nr_of_reviews_ltm'] = df.groupby(['Neighbourhood '])['Calculated_listing_count'].transform(np.sum)
df

,Id,Name,Host_id,Neighbourhood,Room_type,price,Min_nights,Last_review,Calculated_listing_count,Availability_365,Nr_of_reviews_ltm
0,41870,Room in relaxed family house!,182993,Waltham Forest,Private room,37,2,2011-07-02,1,0,5641
1,388306,"Garden flat, Putney",1942893,Wandsworth,Private room,908,1,2012-03-26,1,365,29042
2,306405,Spacious Double Room in East London,1575800,Barking and Dagenham,Private room,41,4,2012-05-27,1,365,2753
3,390876,Peaceful Riverside Island Sanctuary,1954980,Richmond upon Thames,Private room,35,3,2012-05-31,1,0,7212
4,553543,Newly furnished flat in Kensington,2720301,Hammersmith and Fulham,Private room,35,14,2012-07-17,1,0,37516
...,...,...,...,...,...,...,...,...,...,...,...
69346,679489255469079962,LUXURY sky-high suite + RIVER + CITY SKYLINE v...,52259351,Lambeth,Entire home/apt,315,1,NaN,14,365,26835
69347,44639074,Entire lovely family home with garden near London,17343987,Kingston upon Thames,Entire home/apt,110,1,NaN,1,0,7108
69348,38338919,Loveshack baby!,5005089,Richmond upon Thames,Entire home/apt,210,3,NaN,1,0,7212
69349,44197476,"Airy, sunlit, lovely apartment with Thames views",266785086,Tower Hamlets,Private room,96,2,NaN,1,0,49189


In [19]:
#subsetting the columns for the map
view_map = df[['Neighbourhood ', 'Nr_of_reviews_ltm']]

In [20]:
view_map.head()

,Neighbourhood,Nr_of_reviews_ltm
0,Waltham Forest,5641
1,Wandsworth,29042
2,Barking and Dagenham,2753
3,Richmond upon Thames,7212
4,Hammersmith and Fulham,37516


In [21]:
view_map.shape

(69351, 2)

# Data cleaning

In [22]:
#Checking subset for nulls
view_map.isnull().sum()

Neighbourhood        0
Nr_of_reviews_ltm    0
dtype: int64

In [23]:
#The frequency of listing by Neibourhoods
view_map['Neighbourhood '].value_counts(dropna = False)

Westminster               7763
Tower Hamlets             5595
Hackney                   5168
Camden                    4707
Kensington and Chelsea    4612
Southwark                 3968
Islington                 3790
Lambeth                   3719
Wandsworth                3473
Hammersmith and Fulham    2996
Brent                     2068
Lewisham                  2016
Haringey                  1898
Newham                    1682
Barnet                    1611
Ealing                    1553
Greenwich                 1514
Waltham Forest            1322
Merton                    1114
Richmond upon Thames      1108
Hounslow                  1078
Croydon                   1066
Hillingdon                 714
Redbridge                  669
Enfield                    617
Kingston upon Thames       609
Bromley                    594
Harrow                     444
Barking and Dagenham       433
City of London             424
Bexley                     411
Havering                   329
Sutton  

In [24]:
#drop duplicate values
view_map = view_map.drop_duplicates()

In [25]:
view_map.shape

(33, 2)

In [26]:
view_map.sort_values(by = 'Nr_of_reviews_ltm', ascending=False)

,Neighbourhood,Nr_of_reviews_ltm
19,Westminster,174901
44,Kensington and Chelsea,114278
14,Camden,77932
38,Tower Hamlets,49189
27,Southwark,45855
5,Brent,43393
15,Islington,37974
4,Hammersmith and Fulham,37516
6,Hackney,30081
1,Wandsworth,29042


In [52]:
# Setting up a map
map = folium.Map(location = [100, 0], zoom_start = 1.5)


folium.Choropleth(
    geo_data = Country_geo, 
    data = df,
    columns = ['Neighbourhood ', 'Nr_of_reviews_ltm'],
    key_on = 'feature.properties.name', 
    fill_color = 'YlOrBr', fill_opacity=0.5, line_opacity=0.1,
    legend_name = "Nr of views").add_to(map)
folium.LayerControl().add_to(map)

map


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/asus/Documents/Work/Projects/Career Foundry/Data Analysis Immersion - Achievement 6/us-states.json'

In [68]:
f = open(r'C:\Users\Admin\London Uk Airbnb\Original data\custom.geo.json')
data = json.load(f)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 2245: character maps to <undefined>